In [1]:
# imports
from pymongo import MongoClient
import pandas as pd
from funciones import get_divisas, get_values, get_ciudades, geopoint
from collections import Counter

In [2]:
# conexión con  mongo
client = MongoClient("mongodb://localhost:27017/")

In [3]:
# conexión con la base de datos "companies"
db = client.companies

In [4]:
# coleción companies
collection_companies = db.companies

In [5]:
# Query para seleccionar empresas de sectores similares al mío y que sigan en funcionamiento

query = collection_companies.find({"$and": [
                                        {"offices": {"$exists": True}},
                                        {"offices": {"$ne": None}},
                                        {"deadpooled_year": {"$type": 10}},
                                        {"number_of_employees": {"$gte": 20}},
                                        {"$or": [
                                        {"category_code": "games_video"},
                                        {"category_code": "software"},
                                        {"category_code": "web"},
                                        {"category_code": "mobile"},
                                        {"category_code": "social"},
                                        {"category_code": "photo_video"},
                                        {"category_code": "analytics"},
                                        ]},
                                    ]
                                    },
                                    {"_id": 0, "crunchbase_url": 0, "permalink": 0, "homepage_url": 0,
                                     "blog_url": 0, "blog_feed_url": 0, "twitter_username": 0, "founded_month":0,
                                     "founded_day": 0, "deadpooled_month":0, "deadpooled_day": 0, "deadpooled_url":0,
                                     "ipo":0, "tag_list":0, "alias_list":0, "email_address":0, "phone_number":0,
                                     "created_at":0, "updated_at":0, "overview":0, "relationships":0, "competitions":0,
                                     "providerships":0, "funding_rounds":0, "investments":0, "milestones":0,
                                     "products": 0,"acquisition": 0, "acquisitions": 0, "video_embeds": 0, 
                                     "screenshots": 0, "external_links": 0, "partners": 0, "image": 0
                                }
                                )

In [6]:
data_filter = pd.DataFrame(query)

In [7]:
data_filter.head(3)

,name,category_code,number_of_employees,founded_year,deadpooled_year,description,total_money_raised,offices
0,Twitter,social,1300,2006.0,None,Real time communication platform,$1.16B,"[{'description': '', 'address1': '1355 Market ..."
1,Facebook,social,5299,2004.0,None,Social network,$2.43B,"[{'description': 'Headquarters', 'address1': '..."
2,Plaxo,web,50,2002.0,None,Contact Management,$28.3M,"[{'description': 'HQ', 'address1': '1050 Enter..."


In [8]:
data_filter.shape

(1506, 8)

In [9]:
# Limpieza columna offices
# Elimino las filas cuyo valor en "Offices" sea una lista vacía
data_filter = data_filter[pd.Series(map(len, data_filter["offices"])) > 0]

In [10]:
# Creo dos nuevas columnas con las latitudes y las longituedes 
data_filter["latitude"] = [element[0]["latitude"] for element in data_filter["offices"]]
data_filter["longitude"] = [element[0]["longitude"] for element in data_filter["offices"]]

In [11]:
# Elimino aquellas filas cuyo valor para latitude sea None
# ~ invierte valores booleanos
data_filter = data_filter[~ data_filter["latitude"].isna()]

In [12]:
# Elimino los valores duplicados
data_filter = data_filter.drop_duplicates(subset="latitude")

In [13]:
# Creo tres nuevas columnas "country", "state", "city"

In [14]:
data_filter["country"] = [element[0]["country_code"] for element in data_filter["offices"]]
data_filter["state"] = [element[0]["state_code"] for element in data_filter["offices"]]
data_filter["city"] = [element[0]["city"] for element in data_filter["offices"]]

In [15]:
# Limpieza columna "total_money_raised"

In [16]:
currency = [e[0] for e in data_filter["total_money_raised"]]
unique_currency = set(currency)

In [17]:
divisas = {
        "$": "Dolares estadounidenses", 
        "€": "Euros", 
        "C": "Dolares canadienses",
        "C$": "Dolares canadienses",
        "£": "Libras", 
        "kr": "Coronas suecas"
    }

data_filter["currency"] = get_divisas(currency, divisas)

In [18]:
valores = {"B": 1e9, "M": 1e6, "k": 1e3}

In [19]:
data_filter["total_amount_raised"] = data_filter["total_money_raised"].apply(lambda x: get_values(x, divisas, valores))

In [20]:
# Eliminar filas de empresas que hayan ganado menos de un millón
data_filter = data_filter[data_filter["total_amount_raised"] >= 1000000]

In [21]:
# Decido quedarme con las 15 ciudades que más empresas tienen bajo los criterios seleccionados
ciudades = Counter(data_filter["city"])
ciudades = dict([city for city in ciudades.most_common(15)])
ciudades

{'San Francisco': 51,
 'New York': 31,
 'Seattle': 13,
 'Mountain View': 12,
 'Palo Alto': 11,
 'Sunnyvale': 10,
 'Redwood City': 10,
 'Santa Clara': 9,
 'San Mateo': 9,
 'Cambridge': 9,
 'London': 9,
 'San Jose': 7,
 'Paris': 7,
 'Austin': 6,
 'Boston': 6}

In [22]:
ciudades_list = list(ciudades.keys())

In [23]:
data_filter["city"] = data_filter["city"].apply(lambda x: get_ciudades(x, ciudades_list))

In [24]:
data_filter = data_filter[data_filter["city"] != "other"]

In [25]:
# creo la columna geo para los geopoints
data_filter["geo"] = data_filter.apply(lambda x: geopoint(x["longitude"], x["latitude"]), axis = 1)

In [26]:
data_filter.head()

,name,category_code,number_of_employees,founded_year,deadpooled_year,description,total_money_raised,offices,latitude,longitude,country,state,city,currency,total_amount_raised,geo
0,Twitter,social,1300,2006.0,None,Real time communication platform,$1.16B,"[{'description': '', 'address1': '1355 Market ...",37.776805,-122.416924,USA,CA,San Francisco,Dolares estadounidenses,1.160000e+09,"{'type': 'Point', 'coordinates': [-122.4169244..."
2,Plaxo,web,50,2002.0,None,Contact Management,$28.3M,"[{'description': 'HQ', 'address1': '1050 Enter...",37.387845,-122.055197,USA,CA,Sunnyvale,Dolares estadounidenses,2.830000e+07,"{'type': 'Point', 'coordinates': [-122.055197,..."
3,eBay,web,15000,1995.0,None,Online Marketplace,$6.7M,"[{'description': 'Headquarters', 'address1': '...",37.295005,-121.930035,USA,CA,San Jose,Dolares estadounidenses,6.700000e+06,"{'type': 'Point', 'coordinates': [-121.930035,..."
5,Kyte,games_video,40,2006.0,None,Online & Mobile Video Platform,$23.4M,"[{'description': None, 'address1': '442 Post S...",37.788482,-122.409173,USA,CA,San Francisco,Dolares estadounidenses,2.340000e+07,"{'type': 'Point', 'coordinates': [-122.409173,..."
6,Jingle Networks,mobile,35,2005.0,None,Voice and Mobile Search,$88.7M,"[{'description': '', 'address1': '475 Park Ave...",37.480999,-122.173887,USA,NY,New York,Dolares estadounidenses,8.870000e+07,"{'type': 'Point', 'coordinates': [-122.173887,..."


In [27]:
data_filter.shape

(200, 16)

In [28]:
# Limpieza definitiva columnas

In [29]:
data_filter = data_filter.drop(["total_money_raised", "offices"], axis=1)

In [30]:
# Guardar como JSON
data_filter.to_json(r"clean_companies.json", orient="records")